In [1]:
!pip install s3fs

  Using cached s3fs-0.4.2-py3-none-any.whl (19 kB)


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime as dt
import os
import boto3
from sklearn.metrics import mean_absolute_error
from io import StringIO
from sagemaker import get_execution_role
import s3fs

In [3]:
pop_df = pd.read_csv('stock_popularity.csv')

In [4]:
pop_df.set_index('symbol', inplace= True)

In [5]:
pop_df.head()

,users_holding,date
symbol,,
A,1180.0,2018-05-02
A,11143.0,2018-05-03
A,14099.0,2018-05-04
A,14112.0,2018-05-05
A,14112.0,2018-05-06


In [6]:
price_df = pd.read_csv("stock_prices_latest.csv")
price_df.set_index('symbol', inplace = True)
price_df.sort_values(by=['symbol','date'], inplace = True)
price_df.head()

,date,open,high,low,close,close_adjusted,volume,split_coefficient
symbol,,,,,,,,
A,1999-11-18,45.50,50.00,40.00,44.00,29.6303,44739900,1.0
A,1999-11-19,42.94,43.00,39.81,40.38,27.1926,10897100,1.0
A,1999-11-22,41.31,44.00,40.06,44.00,29.6303,4705200,1.0
A,1999-11-23,42.50,43.63,40.25,40.25,27.1050,4274400,1.0
A,1999-11-24,40.13,41.94,40.00,41.06,27.6505,3464400,1.0


In [7]:
merged_df = pd.merge(pop_df, price_df, on=['symbol', 'date'])
merged_df.head()

,users_holding,date,open,high,low,close,close_adjusted,volume,split_coefficient
symbol,,,,,,,,,
A,1180.0,2018-05-02,66.00,66.860,65.77,65.91,65.91,2240482,1.0
A,11143.0,2018-05-03,65.77,66.460,64.86,66.34,66.34,2365851,1.0
A,14099.0,2018-05-04,66.03,67.250,65.54,67.00,67.00,1331017,1.0
A,11718.0,2018-05-07,67.16,67.980,67.07,67.39,67.39,1468735,1.0
A,13851.0,2018-05-08,67.08,67.435,66.84,67.37,67.37,1916103,1.0


In [8]:
corr_df = merged_df.groupby('symbol')[['users_holding','close_adjusted']].corr().iloc[0::2,[-1]]
corr_df.head()

,,close_adjusted
symbol,,
A,users_holding,0.143153
AA,users_holding,-0.806037
AAL,users_holding,-0.641547
AAMC,users_holding,0.438661
AAME,users_holding,-0.255453


In [9]:
#Column has not been renamed, but close_adjusted essentially contains our correlations
high_corr_df = corr_df[(corr_df['close_adjusted'] >= 0.845) | (corr_df['close_adjusted'] <= -0.845)]
high_corr_df.info

<bound method DataFrame.info of                       close_adjusted
symbol                              
AAWW   users_holding       -0.873919
ACER   users_holding       -0.854497
ACMR   users_holding        0.869598
AFH    users_holding       -0.908621
AHPI   users_holding        0.955130
...                              ...
VIIX   users_holding        0.912828
VIIZ   users_holding       -0.912840
YJ     users_holding       -0.855116
YPF    users_holding       -0.873987
ZEAL   users_holding        0.870370

[115 rows x 1 columns]>

In [10]:
high_corr_df = high_corr_df.droplevel(None)

In [11]:
high_corr_df.index

Index(['AAWW', 'ACER', 'ACMR', 'AFH', 'AHPI', 'AMRH', 'ANDE', 'APLT', 'APT',
       'ARCH',
       ...
       'TH', 'TLT', 'TRQ', 'TRXC', 'TUSK', 'VIIX', 'VIIZ', 'YJ', 'YPF',
       'ZEAL'],
      dtype='object', name='symbol', length=115)

In [12]:
merged_df.index

Index(['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',
       ...
       'ZYXI', 'ZYXI', 'ZYXI', 'ZYXI', 'ZYXI', 'ZYXI', 'ZYXI', 'ZYXI', 'ZYXI',
       'ZYXI'],
      dtype='object', name='symbol', length=2491314)

In [13]:
keys = list(high_corr_df.index.values)
cleaned_df = merged_df[(merged_df.index).isin(high_corr_df.index)]

In [14]:
cleaned_df.info

<bound method DataFrame.info of         users_holding        date   open   high      low    close  \
symbol                                                              
AAWW            206.0  2018-05-02  63.90  63.90  62.1500  62.7500   
AAWW           2041.0  2018-05-03  63.30  68.00  63.3000  67.4000   
AAWW           2616.0  2018-05-04  67.50  67.70  65.0000  67.1500   
AAWW           2180.0  2018-05-07  67.25  67.85  67.0500  67.7500   
AAWW           2623.0  2018-05-08  67.40  68.55  67.2500  68.0500   
...               ...         ...    ...    ...      ...      ...   
ZEAL           6290.0  2020-04-09  32.87  33.80  32.8700  33.4200   
ZEAL           6273.0  2020-04-13  33.06  33.06  32.1900  33.0000   
ZEAL           6306.0  2020-04-14  32.80  33.37  32.8000  33.3700   
ZEAL           6331.0  2020-04-15  31.39  31.79  30.9882  31.6148   
ZEAL           6322.0  2020-04-16  33.02  33.02  31.8200  33.0000   

        close_adjusted  volume  split_coefficient  
symbol            

In [15]:
cleaned_df['Pop_EMA'] = cleaned_df.groupby('symbol')['users_holding'].transform(lambda x: x.ewm(span = 5).mean())
cleaned_df.head(20)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,users_holding,date,open,high,low,close,close_adjusted,volume,split_coefficient,Pop_EMA
symbol,,,,,,,,,,
AAWW,206.0,2018-05-02,63.90,63.900,62.150,62.75,62.75,397180,1.0,206.000000
AAWW,2041.0,2018-05-03,63.30,68.000,63.300,67.40,67.40,983735,1.0,1307.000000
AAWW,2616.0,2018-05-04,67.50,67.700,65.000,67.15,67.15,308491,1.0,1927.052632
AAWW,2180.0,2018-05-07,67.25,67.850,67.050,67.75,67.75,172128,1.0,2032.123077
AAWW,2623.0,2018-05-08,67.40,68.550,67.250,68.05,68.05,395837,1.0,2258.952607
AAWW,2629.0,2018-05-09,68.35,69.000,67.900,68.55,68.55,206568,1.0,2394.172932
AAWW,2591.0,2018-05-10,68.80,69.330,68.350,68.45,68.45,181232,1.0,2463.860612
AAWW,2603.0,2018-05-11,68.60,69.200,67.250,68.05,68.05,134526,1.0,2512.123553
AAWW,2613.0,2018-05-14,67.90,68.650,67.150,67.60,67.60,158448,1.0,2546.647071


In [16]:
cleaned_df = cleaned_df[(cleaned_df != 0).all(1)]

In [18]:
compression_opts = dict(method='zip',
                        archive_name='stock_project.csv')  
cleaned_df.to_csv('dataScienceProject.zip', index=True, compression=compression_opts)

In [21]:
cleaned_df.index

Index(['AAWW', 'AAWW', 'AAWW', 'AAWW', 'AAWW', 'AAWW', 'AAWW', 'AAWW', 'AAWW',
       'AAWW',
       ...
       'ZEAL', 'ZEAL', 'ZEAL', 'ZEAL', 'ZEAL', 'ZEAL', 'ZEAL', 'ZEAL', 'ZEAL',
       'ZEAL'],
      dtype='object', name='symbol', length=41564)

In [ ]:
acb_df = df[df.symbol == 'ACB']
drop_cols = ['close_adjusted', 
            'volume', 'split_coefficient']
acb_df.drop(drop_cols, axis=1, inplace=True)
acb_df_ordered = acb_df.sort_values(by=['date'], ascending=False)
acb_df_ordered.head(10)

In [ ]:
plt.figure(figsize=(18,10))
plt.plot(acb_df_ordered.date, acb_df_ordered.close)
plt.title('Closing price of ACB')
plt.ylabel('Closing price ($)')
plt.xlabel('Trading day')
plt.grid(False)
plt.show()

In [ ]:
pop_df = pd.read_csv('ACB.csv', parse_dates=['timestamp'], index_col = ['timestamp'])
pop_df.head()

In [ ]:
pop_df_daily = pop_df.resample('D').sum()
pop_df_daily.index.names = ['date']
pop_df_daily.head()

In [ ]:
acb_df_ordered.index = pd.to_datetime(acb_df_ordered.date, format="%Y-%m-%d", errors='raise')
acb_df_ordered.head()

In [ ]:
acb_df_ordered.drop('date', axis=1, inplace=True)

In [ ]:
acb_df_ordered.head()

In [ ]:
acb_df_ordered.index

In [ ]:
pop_df_daily.index

In [ ]:
combined_df = pd.merge(acb_df_ordered, pop_df_daily, on='date')
combined_df.head()

In [ ]:
fig, ax1 = plt.subplots()
fig.set_figheight(12)
fig.set_figwidth(20)
ax2 = ax1.twinx()
x = combined_df.index

ax1.plot(x, combined_df.close, 'b-')
ax2.plot(x, combined_df.users_holding, 'r-')

ax1.set_xlabel('Trading day')
ax1.set_ylabel('Closing price ($)')
ax2.set_ylabel('Number of users holding (in millions)')